Setup and Data Loading

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Load the dataset
df = pd.read_csv('/home/silver/Healthcare-Chatbot-using-Natural-Language-Processing/clean datasets/Symptom2Disease_cleaned.csv')

[nltk_data] Downloading package punkt to /home/silver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/silver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
import nltk

# Download all required NLTK data
nltk.download('punkt')  # For tokenization
nltk.download('stopwords')  # For stopword removal
nltk.download('wordnet')  # For lemmatization (alternative to stemming)
nltk.download('omw-1.4')  # Open Multilingual WordNet (required for wordnet)

# Verify NLTK data path
print("NLTK data paths:", nltk.data.path)

[nltk_data] Downloading package punkt to /home/silver/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/silver/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/silver/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/silver/nltk_data...


NLTK data paths: ['/home/silver/nltk_data', '/home/silver/Healthcare-Chatbot-using-Natural-Language-Processing/venv/nltk_data', '/home/silver/Healthcare-Chatbot-using-Natural-Language-Processing/venv/share/nltk_data', '/home/silver/Healthcare-Chatbot-using-Natural-Language-Processing/venv/lib/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data']


Text Preprocessing

In [12]:
import pandas as pd
import string
from nltk.tokenize import RegexpTokenizer  # Fallback tokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize components
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
tokenizer = RegexpTokenizer(r'\w+')  # Fallback tokenizer

def robust_preprocess(text):
    try:
        if not isinstance(text, str) or not text.strip():
            return ""
            
        # Convert to lowercase
        text = text.lower()
        
        # Remove punctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        
        # Try NLTK word_tokenize first, fallback to regex tokenizer if it fails
        try:
            tokens = word_tokenize(text)
        except:
            tokens = tokenizer.tokenize(text)
        
        # Remove stopwords and lemmatize
        tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
        
        return ' '.join(tokens)
        
    except Exception as e:
        print(f"Error processing text: {text[:50]}... Error: {str(e)[:100]}...")
        return ""

# Apply preprocessing
df['processed_text'] = df['text'].apply(robust_preprocess)

# Verify results
print("\nSample preprocessing results:")
for i in range(3):
    print(f"\nOriginal {i+1}: {df['text'].iloc[i]}")
    print(f"Processed {i+1}: {df['processed_text'].iloc[i]}")


Sample preprocessing results:

Original 1: i have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. it is red, itchy, and covered in dry, scaly patches.
Processed 1: experiencing skin rash arm leg torso past week red itchy covered dry scaly patch

Original 2: my skin has been peeling, especially on my knees, elbows, and scalp. this peeling is often accompanied by a burning or stinging sensation.
Processed 2: skin peeling especially knee elbow scalp peeling often accompanied burning stinging sensation

Original 3: i have been experiencing joint pain in my fingers, wrists, and knees. the pain is often achy and throbbing, and it gets worse when i move my joints.
Processed 3: experiencing joint pain finger wrist knee pain often achy throbbing get worse move joint


In [13]:
import re
from nltk.corpus import wordnet

# Enhanced medical preprocessing
def medical_preprocess(text):
    try:
        if not isinstance(text, str) or not text.strip():
            return ""
            
        # Convert to lowercase
        text = text.lower()
        
        # Handle medical abbreviations/contractions
        text = re.sub(r"\b(i've|i'm|i'd)\b", "", text)
        
        # Special handling for numeric symptoms
        text = re.sub(r"(\d+)\s*(hours|days|weeks|months|years)", r"\1_\2", text)
        
        # Remove punctuation but keep hyphens in compound terms
        text = re.sub(r"(?<!\w)[.,;:!?](?!\w)", "", text)
        
        # Tokenize with fallback
        try:
            tokens = word_tokenize(text)
        except:
            tokens = tokenizer.tokenize(text)
        
        # Medical stopwords to keep
        medical_keep = {'no', 'not', 'nor', 'very', 'more', 'most', 'less', 'least'}
        stop_words = set(stopwords.words('english')) - medical_keep
        
        # Lemmatize with POS tagging for better accuracy
        def lemmatize_with_pos(word):
            pos = wordnet.ADJ if word.endswith(('ing', 'ed')) else wordnet.NOUN
            return lemmatizer.lemmatize(word, pos=pos)
        
        tokens = [lemmatize_with_pos(word) for word in tokens 
                 if word not in stop_words and len(word) > 2]
        
        return ' '.join(tokens)
        
    except Exception as e:
        print(f"Error processing: {text[:50]}... Error: {str(e)[:100]}...")
        return ""

# Apply enhanced preprocessing
df['med_processed'] = df['text'].apply(medical_preprocess)

# Show comparison
print("\nEnhanced Medical Preprocessing Results:")
for i in range(3):
    print(f"\nOriginal {i+1}: {df['text'].iloc[i]}")
    print(f"Standard: {df['processed_text'].iloc[i]}")
    print(f"Medical: {df['med_processed'].iloc[i]}")


Enhanced Medical Preprocessing Results:

Original 1: i have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. it is red, itchy, and covered in dry, scaly patches.
Standard: experiencing skin rash arm leg torso past week red itchy covered dry scaly patch
Medical: experiencing skin rash arm leg torso past week red itchy covered dry scaly patch

Original 2: my skin has been peeling, especially on my knees, elbows, and scalp. this peeling is often accompanied by a burning or stinging sensation.
Standard: skin peeling especially knee elbow scalp peeling often accompanied burning stinging sensation
Medical: skin peeling especially knee elbow scalp peeling often accompanied burning stinging sensation

Original 3: i have been experiencing joint pain in my fingers, wrists, and knees. the pain is often achy and throbbing, and it gets worse when i move my joints.
Standard: experiencing joint pain finger wrist knee pain often achy throbbing get worse move joint
Med